In [1]:
import numpy as np
import pickle
import cartopy.io.shapereader as shpreader
from cartopy.feature import ShapelyFeature
from shapely.prepared import prep
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd

In [2]:
# some useful arrays to translate between naming conventions

election_year_list = np.array([1992, 1994, 1996, 1998, 2000, 2002, 2004, 2006, 2008, 2010, 2012, 
                                   2014, 2016, 2018])
congress_ID_list = np.array([103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116])

state_names = np.array(['ALABAMA', 'ALASKA', 'ARIZONA', 'ARKANSAS', 'CALIFORNIA', 
               'COLORADO', 'CONNECTICUT', 'DELAWARE', 'FLORIDA', 'GEORGIA', 
               'HAWAII', 'IDAHO', 'ILLINOIS', 'INDIANA', 'IOWA', 'KANSAS', 
               'KENTUCKY', 'LOUISIANA', 'MAINE', 'MARYLAND', 'MASSACHUSETTS', 
               'MICHIGAN', 'MINNESOTA', 'MISSISSIPPI', 'MISSOURI', 'MONTANA', 
               'NEBRASKA', 'NEVADA', 'NEW HAMPSHIRE', 'NEW JERSEY', 'NEW MEXICO', 
               'NEW YORK', 'NORTH CAROLINA', 'NORTH DAKOTA', 'OHIO', 'OKLAHOMA', 
               'OREGON', 'PENNSYLVANIA', 'RHODE ISLAND', 'SOUTH CAROLINA', 
               'SOUTH DAKOTA', 'TENNESSEE', 'TEXAS', 'UTAH', 'VERMONT', 
               'VIRGINIA', 'WASHINGTON', 'WEST VIRGINIA', 'WISCONSIN', 'WYOMING'])

state_abbrs = np.array(['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL',
              'IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT',
              'NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI',
              'SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY'])

In [3]:
# The proj.4 string:
# +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no_defs

In [4]:
year = 2014
congress_ID = '114'
shpfilename = 'districtShapes-{0}/districts{0}.shp'.format(congress_ID)

In [5]:
def read_shapefiles(election_years,verbose=True):
    # read in the standard dictionary 
    # todo: need a way to generalize the directory path
    district_df = pickle.load(open('/Users/Elise/Desktop/ac209_final_project/ac209a_project/Datasets/master_index.p','rb'))
    district_df['shape'] = [np.nan]*district_df.shape[0] # make a blank column
    district_df['shape'] = district_df['shape'].astype(object) # reassign to object so it can hold shapely stuff

    for election_year in election_years:
        # convert election year to "Nth Congress" 
        congress_ID = congress_ID_list[election_year_list==election_year][0]

        # read in the shapefile (must be named 'districtsN.shp' in a folder titled 'districtShapesN')
        shpfilename = 'districtShapes-{0}/districts{0}.shp'.format(congress_ID)
        reader = shpreader.Reader(shpfilename) 
        districts = reader.records() # get full records
        geometries = reader.geometries() # get just the shape

        # put the shapefiles into the standard dictionary
        for record in reader.records(): # loop over districts
            attr = record.attributes # dictionary of information about the district
            poly = record.geometry # coordinates of the district as a shapely polygon

            # get the index of the districts
            if any(state_names==attr['STATENAME'].upper()): # filter out districts that aren't in states
                ST = state_abbrs[state_names==attr['STATENAME'].upper()][0]
            else: # pretty much just Washington, DC
                print('{} is not a state.'.format(attr['STATENAME'].upper()))
                continue
            id_int = int(attr['DISTRICT'])
            if id_int == 0: # todo: do we want to change this back? Looks like 0 for at-large districts is the convention. 
                id_int = 1
            ID = '{0:02d}'.format(id_int)
            ind = '{}_{}_{}'.format(ST, ID, election_year)
            if verbose:
                print('{} was read in.'.format(ind))
            # put the polygon in the dictionary
            district_df.at[ind,'shape'] = poly
    
    return district_df

In [6]:
district_df = read_shapefiles([2010,2012,2014], verbose=False)

DISTRICT OF COLUMBIA is not a state.
DISTRICT OF COLUMBIA is not a state.
DISTRICT OF COLUMBIA is not a state.


In [7]:
# find if a district has changed between last year and this year
def check_if_districts_changed(this_year, district_df, threshold_for_change=0.1):
    # loop over states so you only have to compare districts in-state
    # otherwise, comparing each district to 434 other districts would be super slow
    for ST in state_abbrs: 
        prev_year = this_year-2
        # get the relevant districts
        districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==this_year)]
        districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                                 district_df['year']==prev_year)]
        # loop over districts in your current year
        for ind,district in districts.iterrows():
            # find previous year's district
            district_prev = districts_prev.loc[districts_prev['district']==district['district']]
            ind_prev = district_prev.index

            # determine whether district is new or borders have changed
            if ind_prev.shape[0]==0: # if the district didn't exist last year
                # then this district is new this year
                district_df.loc['border_change',ind] = 'new'
            else: 
                # check if the borders have changed
                shape_prev = district_prev['shape'].values[0]
                shape = district['shape']

                # check if shapes intersect with themselves
                if not (shape.is_valid and shape_prev.is_valid): 
                    # if they do, use buffer to correct this
                    print('The following polygons intersected with themselves. Attempting to buffer shape...')
                    if not shape.is_valid:
                        print(ind)
                        shape = shape.buffer(0)
                    if not shape_prev.is_valid:
                        print(ind_prev)
                        shape_prev = shape_prev.buffer(0)

                # calculate overlap percent
                area = shape.area # area of this district
                area_prev = shape_prev.area
                overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
                frac_overlap = overlap_area/np.max([area,area_prev]) # fractional overlap between new and old district
                # divide by the larger of the old or new district
                
                if (1.-frac_overlap) < threshold_for_change: 
                    # then district has not changed
                    district_df.loc[ind,'border_change'] = 'same'
                else:
                    # district has changed
                    district_df.loc[ind,'border_change'] = 'changed'
                    print(ind)
                    print(frac_overlap)
                    
    return district_df

In [8]:
this_year = 2014
#district_df = check_if_districts_changed(this_year, district_df, threshold_for_change=0.1)

In [ ]:
# compute overlap percent between this district and last year's districts
this_year = 2012
district_df['overlap_frac'] = [np.nan]*district_df.shape[0] # make a blank column
district_df['overlap_frac'] = district_df['overlap_frac'].astype(object) # convert to object 

# loop over states so you only have to compare districts in-state
# otherwise, comparing each district to 434 other districts would be super slow
for ST in state_abbrs: 
    prev_year = this_year-2
    # get the relevant districts
    districts = district_df.loc[np.logical_and(district_df['state']==ST,
                                             district_df['year']==this_year)]
    districts_prev = district_df.loc[np.logical_and(district_df['state']==ST,
                                             district_df['year']==prev_year)]
        
    for ind,district in districts.iterrows(): # loop over districts in your current year
        overlap_dict = {}
        for ind_prev,district_prev in districts_prev.iterrows(): # loop over districts in previous year
            shape_prev = district_prev['shape']
            shape = district['shape']
            
            # check if shapes intersect with themselves
            if not (shape.is_valid and shape_prev.is_valid): 
                # if they do, use buffer to correct this
                print('The following polygons intersected with themselves. Attempting to buffer shape...')
                if not shape.is_valid:
                    print(ind)
                    shape = shape.buffer(0)
                if not shape_prev.is_valid:
                    print(ind_prev)
                    shape_prev = shape_prev.buffer(0)
            
            # calculate frac overlap
            area = shape.area # area of this district
            area_prev = shape_prev.area
            overlap_area = shape.intersection(shape_prev).area # area of overlap between shape and shape_prev
            frac_overlap = overlap_area/area # fractional overlap between new and old district
            
            if frac_overlap > 0.:
                overlap_dict[ind_prev] = frac_overlap

        print(ind)
        print(overlap_dict)
        district_df.at[ind, 'overlap_frac'] = overlap_dict

In [ ]:
# read shapefile
reader = shpreader.Reader(shpfilename) 
districts = reader.records() # get full records
geometries = reader.geometries() # get just the shape

district_df = pickle.load(open('/Users/Elise/Desktop/ac209_final_project/ac209a_project/Datasets/master_index.p','rb'))
district_df['shape'] = [np.nan]*district_df.shape[0] # make a blank column
district_df['shape'] = district_df['shape'].astype(object) # reassign to object so it can hold shapely stuff

record = next(districts) # the record has both shape and geometry
district_geo = next(geometries) # get next shape

In [ ]:
for record in reader.records():
    attr = record.attributes
    poly = record.geometry
    if any(state_names==attr['STATENAME'].upper()): # don't index DC
        ST = state_abbrs[state_names==attr['STATENAME'].upper()][0]
    else: 
        print('{} is not a state.'.format(attr['STATENAME'].upper()))
        continue
    id_int = int(attr['DISTRICT'])
    if id_int ==0:
        id_int = 1
    ID = '{0:02d}'.format(id_int)
    ind = '{}_{}_{}'.format(ST, ID, year)
    print(ind)
        
    district_df.at[ind,'shape'] = poly

In [ ]:
print(district_df[district_df['year']==2014])

In [ ]:
record.geometry # record has geometry
record.attributes # record has attributes
# STATENAME, DISTRICT, STARTCONG, ENDCONG may be useful
# Will want to correlate STARTCONG, ENDCONG to year

In [ ]:
# testing a plot
shape_feature = ShapelyFeature(record.geometry,
                                ccrs.PlateCarree(), edgecolor='black')
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.set_xlim((-125, -67))
ax.set_ylim((24,50))
ax.add_feature(shape_feature, facecolor='blue')
plt.show()